In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

from scipy.stats import entropy


import lib as lib


### Working on breast cancer data

In [2]:
# import some data to play with
#load the breast cancer dataset 
bcan = datasets.load_breast_cancer()
X = bcan.data
y = bcan.target

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)


# normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [3]:
# initialize params1 and params2

params1=[lib.initialize_weights(X_train.shape[1]) for i in range(100)]# a vector of shape 100,4
# call the initialize_weights function above

params2=[np.random.uniform() for i in range(100)]# a vector of shape 100
# use the np.random.uniform() function

# we have a list of 100 weight vectors (params1) and 100 thresholds (params2)
# convert them to array
params1=np.array(params1)
params2=np.array(params2)


print("Shape of params 1 (weights)",params1.shape)
print("Shape of params 2 (thresholds)",params2.shape)

Shape of params 1 (weights) (100, 30)
Shape of params 2 (thresholds) (100,)


In [4]:
z = lib.objective_fn_vector(params1, params2, X_train, y_train)
# Find the global minimum
param1_min = params1[z.argmin()] # use z.argmin()
param2_min = params2[z.argmin()] # use z.argmin()

print("param1_min",param1_min,"param2_min",param2_min)
print(lib.objective_fn(param1_min, param2_min, X_train, y_train))

param1_min [0.42060474 0.20253407 0.79809366 0.3719351  0.25834853 0.06553277
 0.61057891 0.0365416  0.12210604 0.1125368  0.06619985 0.85793365
 0.21534284 0.58716878 0.76528031 0.67603777 0.99827515 0.21971609
 0.47383982 0.34549535 0.30135373 0.17451278 0.88037256 0.86634892
 0.55425824 0.0622579  0.85016508 0.90439031 0.83388989 0.64761713] param2_min 0.662545010821571
0.6643460978641622


In [5]:

# Hyper-parameter of the algorithm
c1 = c2 = 0.1
w1 = np.array([np.random.uniform() for i in range(X_train.shape[1])])
w2 = 0.8 
# Create particles
n_particles = 20
np.random.seed(100)
params1=[lib.initialize_weights(X_train.shape[1]) for i in range(n_particles)] # a vector of shape n_particles,4
# call the initialize_weights function above

params2=[np.random.uniform() for i in range(n_particles)]# a vector of shape n_particles
# use the np.random.uniform() function

params1=np.array(params1)
params2=np.array(params2)

print("params1 shape is ",params1.shape,"params2 shape is ",params2.shape)

params1 shape is  (20, 30) params2 shape is  (20,)


In [6]:
# define velocity of each weight of every particle
V_param1 = [lib.initialize_weights(X_train.shape[1])*0.1 for i in range(n_particles)] # shape is same as params1
# once again can use initialize_weights function

#define velocity of each threshold of every particle
V_param2 = np.array([np.random.uniform()*0.1 for i in range(n_particles)])# shape is same as params2
# once again use np.random.uniform() function

# Initialize objective values
pbest = (params1,params2)
pbest_obj = lib.objective_fn_vector(params1, params2, X_train, y_train)
gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()])
gbest_obj = pbest_obj.min()

print("pbest obj value for 20 particles are as follows",pbest_obj)
print("gbest obj value among all 20 particles is as follows",gbest_obj)
# note that gbest_obj should be the minimim of all pbest_obj

pbest obj value for 20 particles are as follows [0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461
 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461
 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461]
gbest obj value among all 20 particles is as follows 0.6643460978641622


In [7]:
def update(V_param1,V_param2, params1,params2, pbest, pbest_obj, gbest, gbest_obj):
    "Function to do one iteration of particle swarm optimization"
    # these have been already initialized in the previous cells
    
    # Update params
    r11,r12, r2 = np.random.rand(3)
    V_param1=w1*V_param1+c1*r11*(pbest[0] - params1)+ c2*r2*(gbest[0]-params1)
    V_param2=w2*V_param2+c1*r12*(pbest[1] - params2)+ c2*r2*(gbest[1]-params2)    
#     V = w * V + c1*r11*(pbest - params1) + c2*r2*(gbest.reshape(-1,1)-X)
    params1 = params1 + V_param1
    params2 = params2 + V_param2
    
    obj = lib.objective_fn_vector(params1, params2, X_train, y_train)
    for i in range(pbest[0].shape[0]):
        if pbest_obj[i]>=obj[i]:
            
            pbest[0][i]=params1[i] # update pbest[0][i] with value of params1[i]
            pbest[1][i]=params2[i] # update pbest[1][i] 
            pbest_obj[i]=obj[i]    # also update pbest_obj[i]

            
    gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()]) # update gbest to contain the best from params1 and params 2
    gbest_obj = pbest_obj.min() # update gbest to get the minimum of pbest_obj
 


In [8]:
for i in range(100):
    update(V_param1,V_param2, params1,params2, pbest, pbest_obj, gbest, gbest_obj)
print("PSO found best solution at f({})={}".format(gbest, gbest_obj))
print("Global optimal at f({})={}".format([param1_min,param2_min], lib.objective_fn(param1_min, param2_min, X_train, y_train)))


PSO found best solution at f((array([3.65570865, 7.07190017, 4.05500191, 9.26377692, 0.02294058,
       0.46981782, 2.08550332, 6.37314321, 0.67221797, 5.70213395,
       0.8938173 , 3.40942058, 1.3159591 , 1.04214977, 8.11979007,
       3.3337176 , 3.97242406, 0.74803574, 8.99317535, 1.32078481,
       4.58626422, 4.65795375, 1.97023155, 3.7902729 , 0.66758815,
       1.37132512, 3.22524034, 7.44357446, 4.24719836, 0.85780517]), 0.4098464718143954))=0.6643460978641622
Global optimal at f([array([0.42060474, 0.20253407, 0.79809366, 0.3719351 , 0.25834853,
       0.06553277, 0.61057891, 0.0365416 , 0.12210604, 0.1125368 ,
       0.06619985, 0.85793365, 0.21534284, 0.58716878, 0.76528031,
       0.67603777, 0.99827515, 0.21971609, 0.47383982, 0.34549535,
       0.30135373, 0.17451278, 0.88037256, 0.86634892, 0.55425824,
       0.0622579 , 0.85016508, 0.90439031, 0.83388989, 0.64761713]), 0.662545010821571])=0.6643460978641622


In [10]:
max_tree_size=128
all_optimized_weights_list=[None for i in range(max_tree_size)]
all_optimized_thresh_list=[None for i in range(max_tree_size)]
all_dataset_sizes_list=[None for i in range(max_tree_size)]
all_IG_list=[None for i in range(max_tree_size)]

In [12]:
import numpy as np

def calc_information_gain(y,split):
    '''
    Calculate the information gain of a given split
    
    Parameters:
        y (numpy array): The labels of the dataset
        split (numpy array): A boolean array indicating the split
    
    Returns:
        float: The information gain of the given split
    '''
    n=len(y)
    n1=split.sum()
    n2=n-n1
    p1=n1/n
    p2=1-p1
    H_y=-p1*np.log2(p1+1e-10)-p2*np.log2(p2+1e-10)
    if n1==0 or n2==0:
        return 0
    y1=y[split]
    y2=y[~split]
    H_y1 = calc_entropy(y1)
    H_y2 = calc_entropy(y2)
    IG = H_y - n1/n*H_y1 - n2/n*H_y2
    return IG

def calc_entropy(y):
    '''
    Calculate the entropy of a given dataset
    
    Parameters:
        y (numpy array): The labels of the dataset
    
    Returns:
        float: The entropy of the given dataset
    '''
    n=len(y)
    if n==0:
        return 0
    p1=(y==1).sum()/n
    p2=1-p1
    H_y=-p1*np.log2(p1+1e-10)-p2*np.log2(p2+1e-10)
    return H_y


In [13]:
def find_best_params(train_x,train_y,test_x,test_y,node_number):
    '''
    recursive function to get the best set of weights
    '''
    print("node_number",node_number,"data shape",train_x.shape)
    # exit condition 1: if the node_number is more than the maximum tree size, return
    if node_number>=max_tree_size:
        return
    # exit condition 2: if the training dataset has one or less rows, return 
    if train_x.shape[0]<=1:
        return 
    # exit condition 3: if the train_y has values from only one class (only 0s or only 1s and so on)
    if len(set(train_y))==1:
        return

    # use the initialized lists as global
    global all_optimized_weights_list
    global all_optimized_thresh_list
    global all_dataset_sizes_list
    global all_IG_list

    # Hyper-parameter of the algorithm
    c1 = c2 = 0.1
    w1 = np.array([np.random.uniform() for i in range(train_x.shape[1])])
    w2 = 0.8 

    # Create particles
    n_particles = 20
    np.random.seed(100)
    params1=[lib.initialize_weights(train_x.shape[1]) for i in range(n_particles)]
    params2=[np.random.uniform() for i in range(n_particles)]
    params1=np.array(params1)
    params2=np.array(params2)

    # define velocity of each weight of every particle
    V_param1 = [lib.initialize_weights(train_x.shape[1])*0.1 for i in range(n_particles)]

    # define velocity of each threshold of every particle
    V_param2 = np.array([np.random.uniform()*0.1 for i in range(n_particles)])

    # Initialize objective values
    pbest = (params1,params2)
    pbest_obj = lib.objective_fn_vector(params1, params2, train_x, train_y)
    gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()])
    gbest_obj = pbest_obj.min()

    for i in range(100):
        update(V_param1,V_param2, params1,params2, pbest, pbest_obj, gbest, gbest_obj)
    
    # add the achieved optimized values to the lists
    all_optimized_weights_list[node_number]=gbest[0]
    all_optimized_thresh_list[node_number]=gbest[1]
    all_dataset_sizes_list[node_number]=train_x.shape[0]
    all_IG_list[node_number]=calc_information_gain(train_y,train_y>=gbest[1])
    
    new_ys=np.dot(train_x,gbest[0])
    
    # normalize the new_ys
    new_ys=(new_ys-np.min(new_ys))/(np.max(new_ys)-np.min(new_ys))
    
    # chop the data into two parts: left
    train_x_left=train_x[new_ys>=gbest[1]]
    train_y_left=train_y[new_ys>=gbest[1]]
    left_child_node_num=node_number*2+1
    
    # chop the data into two parts: right
    train_x_right=train_x[new_ys<gbest[1]]
    train_y_right=train_y[new_ys<gbest[1]]
    right_child_node_num=node_number*2+2
    
    # exit condition 4: return if information gain is 0
    if all(ig == 0 for ig in all_IG_list):
        return
    
    print("Left",train_x_left.shape)
    print("Right",train_x_right.shape)
    # make the recursion call for left
    find_best_params(train_x_left,train_y_left,test_x,test_y,left_child_node_num)
    # make the recursion call for right
    find_best_params(train_x_right,train_y_right,test_x,test_y,right_child_node_num)    
    
    
    
    
    

In [14]:
node_number=0
find_best_params(X_train,y_train,X_test,y_test,node_number)

node_number 0 data shape (381, 30)
Left (93, 30)
Right (288, 30)
node_number 1 data shape (93, 30)
Left (19, 30)
Right (74, 30)
node_number 3 data shape (19, 30)
node_number 4 data shape (74, 30)
Left (31, 30)
Right (43, 30)
node_number 9 data shape (31, 30)
Left (9, 30)
Right (22, 30)
node_number 19 data shape (9, 30)
Left (4, 30)
Right (5, 30)
node_number 39 data shape (4, 30)
Left (2, 30)
Right (2, 30)
node_number 79 data shape (2, 30)
Left (1, 30)
Right (1, 30)
node_number 159 data shape (1, 30)
node_number 160 data shape (1, 30)
node_number 80 data shape (2, 30)
node_number 40 data shape (5, 30)
node_number 20 data shape (22, 30)
node_number 10 data shape (43, 30)
Left (28, 30)
Right (15, 30)
node_number 21 data shape (28, 30)
Left (18, 30)
Right (10, 30)
node_number 43 data shape (18, 30)
Left (12, 30)
Right (6, 30)
node_number 87 data shape (12, 30)
node_number 88 data shape (6, 30)
Left (3, 30)
Right (3, 30)
node_number 177 data shape (3, 30)
node_number 178 data shape (3, 30)


In [15]:
X_train.shape

(381, 30)

In [16]:
all_optimized_thresh_list

[0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 None,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 None,
 None,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 None,
 None,
 None,
 None,
 None,
 0.4098464718143954,
 None,
 0.4098464718143954,
 None,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 0.4098464718143954,
 None,
 None,
 None,
 0.4098464718143954,
 None,
 None,
 None,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 None,
 0.4098464718143954,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 0.4098464718143954,
 None,
 None,
 None,

In [17]:
thresh=all_optimized_thresh_list[0]